In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
import math
import random

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

# Multiple Regression

Doing a simple linear regression is cool, but for this to be really useful we want to be able to use a bunch of factors to make a prediction. The process is largely the same, we just have more variables on the input (X) side. It is also difficult/impossible to visualize, since we'd need to see in 3+ dimensions.  

## Exploratory Data Analysis

First, we'll get some data, and look at what the target and the features are going to be. This is a really simplifed example of some EDA that we normally do in advance of machine learning. The first step to any predictive model building is to look at our data and see if anything should be done prior to modelling. This is pretty vague, but we are basically looking to see if there's anything in the data that might be suboptimal for the predictions we want to create. Some common things to look for are:
<ul>
<li> Errors - anything that is just "wrong", words in numeric columns, mislabeled values, etc...
<li> Outliers - values too large or too small. 
<li> Weird Distributions - this is very open-ended, but think of something like a numerical feature where the variance is exceeding low. If every row of the data has the same value, it probably won't help in making predictions, which is inheirently seeking to discriminate. For example, some schools in Japan used to force students to dye their hair black if it wasn't already; if hair color was a feature in your dataset, it wouldn't really add any value if 99.9% of rows of data is "black". 
<li> Missing Values - how should we deal with it if some values are blank?
</ul>

This isn't a conclusive list, and as we explore we may find other things to change or remove. The amount of work we may need to do can also vary widely - if the data is comming from something like a PoS system it will probably be pretty clean and not require many changes; if the data is coming from scraping social media sites from that people fill out, the data may well require massive amounts of cleaning. 

In [ ]:
#Get data
#I'll drop density to make it more realistic.
df = pd.read_csv("data/bodyfat.csv")
df = df.drop(columns={"Density"})
df.head()

Cool. We'll predict the BodyFat (which is hard to accurately measure) by using the other stuff, which is easier to measure. 

Before we start, we want to do some exploration of the data.

In [ ]:
#Start Exploring the Data
df.describe()

At a glance, it looks like there aren't a lot of issues. One thing I notice is that there's a BodyFat reading of 0 for the min. That isn't possible, so we'll need to get rid of that. 

In [ ]:
#Get variable info. 
df.info()

All of our variables are correctly identified as numbers. Often if there's some erroneous data in a column it might get miscategorized as an object. Since we'll be dealing with non-numeric data in ML, starting next workbook, we probably want to double check.  

In [ ]:
#Visual correlation and distributions
sns.pairplot(df)

It appears there are some outliers that probably aren't helping - e.g. Height. 

##### Check for Missing Values

In [ ]:
#Are there nulls?
df.isnull().sum().sort_values()

##### Filter Outliers

In [ ]:
# Remove some outliers
# These were judgement cals. 
# E.g. it is theoretically possible for someone to have fat < 5%, but that's really only people like
# bodybuilders at a competition time. Those types of extreme outliers probably wont' help our model. 
df = df[df["BodyFat"] > 5]
df = df[df["Height"] > 40]
df = df[df["Weight"] < 300]
df = df[df["Ankle"] < 30]
df.describe()

##### Examine Distributions Post-Outliers

In [ ]:
#Check with cleaned data. 
sns.pairplot(df)

Looks pretty good. We may want to examine some of the outliers a bit more closely, but overall the data looks pretty usable. Distributions are all pretty normal. Large outliers are gone. Nothing really jumps out as being a problem.

There does appear to be a large amount of correlation between variables. We'll worry about that later on, for now, looks good. Time to start regressing...


## Setup the data

The basic idea here is the same as one variable regression, only instead of one X, we have a bunch. We can't do this with simple functions, at least not easily, so we'll use a package. 

### Train-Test Split

One thing that can cause an issue for us is with the evaluation of the model's accuracy (more in other models than in simple linear regression) is derived from the data we used to create the model. In reality, we want to have an idea of how accurately our model will make predictions on <i>new</i> data that they've never seen before. To accomplish this, we can get a better idea of the quality of our model by splitting the data into two halves - a training part and a testing part. 

The train_test_split function does this for us in an easy way. It will take in the X and y parts of our original dataset and return both a training and testing array of data for both X and y. So we will train our model with the training data, usually about 70%-80% of the data, then after that model is created we'll test the accuracy on the other ~25% of the data that we held aside for the test set. The results of our model's performance on this "new" data will be more representative of the performance we could expect in normal usage, as this data is new as far as the model is concerned. 

This approach prevents data leakage, meaning that we aren't testing a model on it predicting things it has already seen. We want to remove any possibility of the model "memorizing" the data we gave it to train, and giving better performance than we can expect on real new data. Note that this is much less obvious on linear regression than it is with other model types that can be more complex. The difference on linear regression is minimal, the difference with a tree-based model or a neural network model will be much more dramatic. 

### SciKitLearn Data Setup

We can now prepare the data, and put it into its final shape. We need to generate two arrays, one for X and one for y:
<ul>
<li> X - the features. Should be # of records tall x # of features wide. </li>
<li> y - the target. Should be # of records tall x 1 column wide. </li>
</ul>

This is also the last time that we'll manipulate the data manually - once we create the X/y datasets, then feed those into the train-test split function, we will input that data directly to the ML algorithm as-is. So any data preparation stuff that we want to do ahead of time should be done beforehand. 

This part of the data setup will also be largely identical in all of our examples - generate X/y, do the train-test split, feed into ML algorithm. 

In [ ]:
#Y data is simple, do that first
y = np.array(df["BodyFat"]).reshape(-1,1)
y.shape

Y is identical to the single variable stuff. For X, we have a width to our array. We can take a look again to see what we expect. 

In [ ]:
#Get a new df with only the features we'll use
df_ = df.drop(columns={"BodyFat"})
df_.head()

In [ ]:
#Make that df into an array. 
x = np.array(df_)
x.shape

In [ ]:
#shapes
print("X shape", x.shape)
print("Y shape", y.shape)

### SKL Regress...

Shape for this one looks good! We have 240 rows, which matches the Y. We have 13 columns, which is what we get if we were to count up the columns by hand. Success!!

Time for regression stuff. 

In [ ]:
#Setup
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
from sklearn.model_selection import train_test_split

#### Split Data

In [ ]:
#Split data
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=.3)

##### Train Model

In [ ]:
#Generate model 
model = LinearRegression().fit(xTrain,yTrain)

##### Check Results

We can check how our model performs on the testing data - the 30% of data that we held aside from the model while it was being trained. 

In [ ]:
#Get some info on our new regression model
r_sq = model.score(xTest, yTest)
print('R-squared:', r_sq)

#### View "Slopes" and Intercept

In single regression our model was made up of the slope, and intercept values - we can plug any X in and calculate our prediction. In multiple regression, the results are the same, there's just more of them. We still have one intercept, but now each X has it's own m. The entire equation ends up just having a bunch of "X" terms:

$ y = m1 * x1 + m2 * x2 + m3 * x3 ... + b $

We can extract the values just as we did before, we could also make a manual model to generate predictions - our equation would just be longer than the y = mx + b ones that we created earlier. 

#### Visualizing Multiple Regression

The idea of how multiple regression works is the same, it is just harder to visualize. If we try to look at an example with 2 features (so 3 total values, with the target) and picture it a little. In a single regression we make a prediction by doing a "lookup" in one dimension - the X value is our input, we slide along the X axis until we find that value, then look up to the regression line to see what the target is at that point. In a 2 feature regression, the same idea applies, only we do the lookup with two X values on a plane and the prediction is where they intersect on the 3rd axis:

![3D Regression](images/3d_regression.png "3D Regression")

Now our model, or our line of best fit, is a plane - a two dimensional line. Each feature we add ups that number of dimensions by 1, and it is always 1 less than the total number of dimensions. In a single regression teh value on one axis predicts the other axis, in a multiple regression the value on [# of features] axis, where they interesect, predicts the value on the other axis. Same, same. 

Going up in features, the same idea always applies, we just don't have a timecube to draw it. If we had 13 features like we do above, we'd have a scatter plot in 14 dimensional space, and we'd look to the intercept of the 13 features, which would give us our prediction on the 14th axis - the one belonging to the target. 

In [ ]:
#Our coefficent/slope is now an array of values - one per X. 
#Visualizing the regression would be a 14D space, where these are the slopes in each dimension. 
print('Intercept:', model.intercept_[0])
print('Coefs:', model.coef_[0])

##### Error Metrics

Get the residuals to calculate RMSE. 

The residual stuff is the same no matter how many Xs we have on the input side, since all we are checking is the Y values. 

In [ ]:
#Get RMSE
tmp = model.predict(xTest)
mean_squared_error(tmp, yTest, squared=False)

##### Residuals

We don't generally need to view the residuals if we aren't doing things by hand, the library functions allow us to just look at the summary statistics like RMSE. We can view them for fun, just to confirm they exist. 

In [ ]:
#Get Residuals and picture them in a DF for easy reading. 
tmp1 = pd.DataFrame(yTest, columns={"Y values"})
tmp2 = pd.DataFrame(tmp, columns={"Predictions"})
tmp3 = pd.DataFrame((yTest-tmp), columns={"Residual"})
resFrame = pd.concat([tmp1,tmp2,tmp3], axis=1)
resFrame.head()

## Exercise

Predict the Density value with a sklearn linear regression. 

In [149]:
# Load data and remove bodyfat
dfE = pd.read_csv("data/bodyfat.csv")
dfE.drop(columns={"BodyFat"}, inplace=True)
dfE.head()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [155]:
#Setup Data
yE = np.array(dfE["Density"]).reshape(-1,1)
xE = np.array(dfE.drop(columns={"Density"}))
xE.shape, yE.shape

((252, 13), (252, 1))

In [156]:
#Split and Train
xTrainE, xTestE, yTrainE, yTestE = train_test_split(xE,yE,test_size=.3)
#Generate model 
modelE = LinearRegression().fit(xTrainE,yTrainE)

In [157]:
#Score 
#Get RMSE and R2
tmpE = modelE.predict(xTestE)
rmseE = mean_squared_error(tmpE, yTestE, squared=False)
r_sqE = modelE.score(xTestE, yTestE)

print("RMSE:", rmseE)
print('R-squared:', r_sqE)

RMSE: 0.011128385792870505
R-squared: 0.6505217281635629
